In [ ]:
# src/export
"""
Export model, evaluation result, and training history for version control and possible revert/retraining.
Required mounting to drive and import of importnb to work with .ipynv files. Assumed to have already been done in main script.
Required to be in right directory. also assumed to have been done in main script.

Function list:
-------
1. save_model_and_metrics: Comprehensive export which include model, training history, test metrics, classification report, confusion matrix, and config file
"""
import os
import json
import numpy as np
import pandas as pd
import datetime
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras

# pulling variables from config
import json

with open('config.json') as f:
    cfg = json.load(f)

export_dir = cfg["export"]["export_dir"]
model_name = cfg["export"]["model_name"]

# functions

def save_model_and_metrics(model, history, x_test, y_test, export_dir, model_name, config, note=None):
    """
    Comprehensive export which include model, training history, test metrics, classification report, confusion matrix, and config file
    """

    # Checking and generating an appropriate folder to which to export
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    version = 1
    while True:
        save_path = os.path.join(export_dir, f"{model_name}_v{version:03d}_{timestamp}")
        if not os.path.exists(save_path):
            break
        version += 1
    os.makedirs(save_path)

    # Save the model (includes optimizer state)
    model.save(os.path.join(save_path, f"{model_name}.keras"))

    # Save training history
    pd.DataFrame(history.history).to_csv(os.path.join(save_path, "training_history.csv"), index=False)

    # Save test metrics
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    metrics = {"test_loss": float(loss), "test_accuracy": float(acc)}
    with open(os.path.join(save_path, "test_metrics.json"), "w") as f:
        json.dump(metrics, f, indent=4)

    # Make predictions on the test set
    y_pred = model.predict(x_test)

    # Convert the predicted labels to continuous-multioutput format
    y_pred = np.round(y_pred)

    # Save classification report
    report = classification_report(y_test, y_pred)
    with open(os.path.join(save_path, "classification_report.txt"), "w") as f:
        f.write(report)

    # Save confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    np.savetxt(os.path.join(save_path, "confusion_matrix.csv"),
               cm, delimiter=",", fmt="%d")

    # Save the config file you used
    with open(os.path.join(save_path, "config.json"), "w") as f:
        json.dump(config, f, indent=4)

    # Save note
    if note is not None:
      with open(os.path.join(save_path, "note.txt"), "w") as f:
          f.write(note)

    print(f"Saved model, metrics and config to {save_path}")